In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from service.Classifier.DataLoader.DataLoader import DataLoader
from service.Classifier.Enums.priority import Priority
from service.Classifier.DataLoader.P2DataLoader import P2DataLoader
from service.Classifier.DataLoader.P3DataLoader import P3DataLoader
from service.Classifier.DataLoader.P4DataLoader import P4DataLoader
from service.Classifier.Model.XGBRegressionModel import XGBRegressionModel
from service.Classifier.PreProcessing.RegressionModelPreProcessor import RegressionModelPreProcessor
from service.Classifier.DataSplit.TrainTestSplit import TrainTestSplit
from service.Classifier.Enums.dataTypeEnum import DataType

/Users/fjirigesi/Documents/OOSLAClassifier/service/Classifier/Model/XGBRegressionModel.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  from pandas import np


In [2]:
def initializeByPriority(priority):
    # load new updated data via Features class
    if priority == Priority.P2.value:
        return P2DataLoader()
    elif priority == Priority.P3.value:
        return P3DataLoader()
    elif priority == Priority.P4.value:
        return P4DataLoader()
    return None

In [3]:
priority = Priority.P2.value
def extract_data(priority):
    dataLoader = initializeByPriority(priority)
    data_df = dataLoader.loadTrainingRawData()
    # split data into train test 4:1
    dataSplit = TrainTestSplit()
    train_df, test_df = dataSplit.split(data_df)

    # generate features
    train_df, feature_names = dataLoader.transformRawDataToFeatures(train_df, DataType.TRAINDATA.value)
    test_df, feature_names = dataLoader.transformRawDataToFeatures(test_df, DataType.VALIDATION.value)
    # pre-processing the data based on model type
    preprocessor = RegressionModelPreProcessor(feature_names)
    train_X, train_y, test_X, test_y = preprocessor.preprocessing(train_df, test_df)
    
    return (train_X, train_y, test_X, test_y, train_df, test_df)

In [4]:
import xgboost as xgb
import numpy as np
import pandas as pd
from service.Classifier.Enums.LabelEnum import LabelEnum
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from service.Classifier.PreProcessing.Utils.dataValidator import DataValidator

def evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df):

    model = xgb.XGBRegressor(max_depth=5, n_estimators=100, learning_rate=0.05).fit(train_X, train_y)
    predictions = np.rint(model.predict(test_X)).astype(np.int64)
    submission = pd.DataFrame({'ID': test_df['keyID'],
                               LabelEnum.CLOSEDDAY.value: predictions, 'Actual': test_y})
    rmse = np.sqrt(mean_squared_error(predictions, test_df[LabelEnum.CLOSEDDAY.value]))
    print("RMSE: %f" % rmse)
    mae = mean_absolute_error(predictions, test_y)
    print("MAE: %f" % mae)
    DataValidator().valid(submission, priority, test_df)
    print(confusion_matrix(submission["truth"], submission["prediction"]))
    print(classification_report(submission["truth"], submission["prediction"]))
    
    return (test_y, predictions, submission["truth"], submission["prediction"])

In [5]:
priority = Priority.P2.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p2_regression_true, p2_regression_pred, p2_classification_true, p2_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: NATBEMCR
The project is not in embedding: PX
The project is not in embedding: GGRIP
The project is not in embedding: PROBLEM
The project is not in embedding: CSCNPLAT
RMSE: 9.492367
MAE: 7.596651
[[218 154]
 [ 67 218]]
              precision    recall  f1-score   support

       False       0.76      0.59      0.66       372
        True       0.59      0.76      0.66       285

    accuracy                           0.66       657
   macro avg       0.68      0.68      0.66       657
weighted avg       0.69      0.66      0.66       657



In [6]:
train_df

,keyID,priority,projectCoverage,ResolvedDay,ClosedDay
0,ONBAML-12719,P2,0.049914,12,12
1,RES-20723,P2,0.008016,10,15
2,BILLING-16571,P2,0.038925,13,15
3,ONBAML-12727,P2,0.049914,12,12
4,REGSTRN-12121,P2,0.150463,22,26
...,...,...,...,...,...
2632,ONBAML-12769,P2,0.224657,10,10
2633,COMS-7986,P2,0.008488,12,12
2634,ONBAML-13038,P2,0.049914,11,11
2635,BILLING-16593,P2,0.035085,13,13


In [8]:
priority = Priority.P3.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p3_regression_true, p3_regression_pred, p3_classification_true, p3_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: RISKVRTB
The project is not in embedding: MPPMO
The project is not in embedding: NRP
The project is not in embedding: PROBLEM
The project is not in embedding: WEBRES
The project is not in embedding: WARROOM
RMSE: 19.226565
MAE: 13.450292
[[531  84]
 [132 108]]
              precision    recall  f1-score   support

       False       0.80      0.86      0.83       615
        True       0.56      0.45      0.50       240

    accuracy                           0.75       855
   macro avg       0.68      0.66      0.67       855
weighted avg       0.73      0.75      0.74       855



In [9]:
train_df

,keyID,priority,projectCoverage,ResolvedDay,ClosedDay
0,VOD-3492,P3,0.004128,45,54
1,BILLING-16793,P3,0.035085,48,48
2,USERCRUD-10233,P3,0.018408,49,53
3,RETURNS-15941,P3,0.004985,42,42
4,BILLING-16491,P3,0.035085,32,32
...,...,...,...,...,...
3505,BILLING-16639,P3,0.038925,47,47
3506,RETURNS-16158,P3,0.004128,48,48
3507,VOD-3604,P3,0.251243,69,71
3508,CANCEL-6130,P3,0.004836,43,48


In [10]:
priority = Priority.P4.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p4_regression_true, p4_regression_pred, p4_classification_true, p4_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: SDOCATOP
The project is not in embedding: VATSEC
RMSE: 24.214687
MAE: 16.606383
[[276  62]
 [ 70  62]]
              precision    recall  f1-score   support

       False       0.80      0.82      0.81       338
        True       0.50      0.47      0.48       132

    accuracy                           0.72       470
   macro avg       0.65      0.64      0.65       470
weighted avg       0.71      0.72      0.72       470

